# Azure OpenAI bring your own data api test

- gpt-35-turbo-0601 version is not supported.
```
"Unsupported Model. Model Name: 'gpt-35-turbo' Model Version '0613'. Please retry with supported model: gpt35-turbo-16k, gpt-turbo-32k or gpt4"
```
- vector search supported.

```
queryType = simple, semantic, vector, vectorSimpleHybrid, vectorSemanticHybrid
```
- embedding API cannot access if private endpoint is enabled. (allow all network access) 
```
"Gateway cannot authenticate upstream services. Please contact Microsoft for help."
```

### REST API

https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#completions-extensions


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import requests
import json
import os

AOAI_KEY = os.environ['AOAI_KEY']
AOAI_ENDPOINT = os.environ['AOAI_ENDPOINT']
#MODEL = 'gpt-35-turbo-16k'
MODEL = 'gpt-35-turbo'
#MODEL = 'gpt-4'

EMBED_AOAI_KEY = os.environ['JP_AOAI_KEY']
EMBED_AOAI_ENDPOINT = os.environ['JP_AOAI_ENDPOINT']

EMBED_MODEL = 'text-embedding-ada-002'
VERSION = '2023-08-01-preview'

AZSCH_ENDPOINT = os.getenv("AZSCH_ENDPOINT")  
AZSCH_KEY = os.getenv("AZSCH_KEY")  

Message format is different for each API version
- 2023-06-01-preview
- 2023-08-01-preview


### embedding endpoint
See this this [link](https://github.com/microsoft/sample-app-aoai-chatGPT/tree/ea7a94a7979fc62f56ffac553401d483ff6d807e/scripts#optional-add-vector-embeddings) 

In [3]:
AOAI_EMBEDIING_ENDPOINT=f'{EMBED_AOAI_ENDPOINT}openai/deployments/{EMBED_MODEL}/embeddings?api-version=2023-05-15'

In [4]:
AOAI_CHAT_ENDPOINT = f'{AOAI_ENDPOINT}openai/deployments/{MODEL}/extensions/chat/completions?api-version={VERSION}'

In [5]:
import requests
import json
import sseclient
import os
import time
from datetime import datetime
import sys

def aoai_request(arg, index_name='appsvc'):
    try:
        url = AOAI_CHAT_ENDPOINT
        headers = {
            'Content-Type': 'application/json',
            'api-key': AOAI_KEY
        }
        body = {
            "dataSources": [
                {
                    "type": "AzureCognitiveSearch",
                    "parameters": {
                        "endpoint": AZSCH_ENDPOINT,
                        "key": AZSCH_KEY,
                        "indexName": index_name,
                        "topNDocuments": 2,
                        "inScope": True,
                        "queryType": "vectorSimpleHybrid",
                        "embeddingEndpoint": AOAI_EMBEDIING_ENDPOINT,
                        "embeddingKey": EMBED_AOAI_KEY,

                    }
                }
            ],
          "messages": [
              {"role": "user", "content": arg}
          ],
          "max_tokens": 1000,
          "temperature": 0,
          "top_p": 0.1
        }


        request = requests.post(url, headers=headers, json=body)

        return json.loads(request.text)
    except Exception as ex:
        return ex
    

In [8]:
from IPython.display import display, Markdown, Latex, HTML, JSON

# 2023-08-01-preview
def qna(query, index_name='appsvc'):
    ret = aoai_request(query, index_name)
    
    js = json.loads(ret['choices'][0]['message']['context']['messages'][0]['content'])
    print('intent:', js['intent'])
    
    answer = ret['choices'][0]['message']['content']
    answer = answer.replace("[doc1]", f"[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/{js['citations'][0]['filepath']})")
    answer = answer.replace("[doc2]", f"[2](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/{js['citations'][1]['filepath']})")
    
    print('answer:', answer)
        
    return js, answer

In [7]:
%%time

query = 'What is the App Service?'
js, md = qna(query)

intent: ["What is Azure App Service?", "Azure App Service definition", "Azure App Service explanation"]
answer: Azure App Service is a fully managed HTTP-based service for hosting web applications, REST APIs, and mobile back ends[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/overview.md). It supports multiple languages and frameworks, including .NET, .NET Core, Java, Node.js, PHP, and Python, and can run on both Windows and Linux-based environments[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/overview.md). App Service provides various features such as security, load balancing, autoscaling, and automated management, and offers DevOps capabilities such as continuous deployment from Azure DevOps, GitHub, Docker Hub, and other sources, package management, staging environments, custom domain, and TLS/SSL certificates[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/overview.md). The compute resourc

In [9]:
display(Markdown(md))

Azure App Service is a fully managed HTTP-based service for hosting web applications, REST APIs, and mobile back ends[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/overview.md). It supports multiple languages and frameworks, including .NET, .NET Core, Java, Node.js, PHP, and Python, and can run on both Windows and Linux-based environments[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/overview.md). App Service provides various features such as security, load balancing, autoscaling, and automated management, and offers DevOps capabilities such as continuous deployment from Azure DevOps, GitHub, Docker Hub, and other sources, package management, staging environments, custom domain, and TLS/SSL certificates[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/overview.md). The compute resources used by App Service are determined by the App Service plan, which defines a set of compute resources for a web app to run, including the operating system, region, number and size of VM instances, and pricing tier[2](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/overview-hosting-plans.md). The pricing tier of an App Service plan determines the App Service features and the cost[2](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/overview-hosting-plans.md).

In [11]:
%%time

query = 'Does App Service support Java?'
js, md = qna(query)
display(Markdown(md))

intent: ["Does App Service support Java?", "Java support in Azure App Service"]
answer: Yes, Azure App Service supports Java for building, deploying, and scaling Java SE, Tomcat, and JBoss EAP web applications on a fully managed service[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/configure-language-java.md)[2](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/configure-language-java.md). Java developers can deploy applications with Maven plugins, from the command line, or in editors like IntelliJ, Eclipse, or Visual Studio Code[2](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/configure-language-java.md). Azure App Service also provides automatic patching and fixes for supported JDKs on a quarterly basis, as well as security updates for major vulnerabilities[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/configure-language-java.md). However, Tomcat 8.0 has reached end o

Yes, Azure App Service supports Java for building, deploying, and scaling Java SE, Tomcat, and JBoss EAP web applications on a fully managed service[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/configure-language-java.md)[2](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/configure-language-java.md). Java developers can deploy applications with Maven plugins, from the command line, or in editors like IntelliJ, Eclipse, or Visual Studio Code[2](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/configure-language-java.md). Azure App Service also provides automatic patching and fixes for supported JDKs on a quarterly basis, as well as security updates for major vulnerabilities[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/configure-language-java.md). However, Tomcat 8.0 has reached end of life and will not receive security updates on Azure App Service, so it is recommended to migrate to Tomcat 8.5 or 9.0 if possible[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/configure-language-java.md). Additionally, community support for Java 7 will terminate on July 29, 2022, and Java 7 will be retired from App Service at that time[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/configure-language-java.md).

CPU times: user 11.3 ms, sys: 509 µs, total: 11.8 ms
Wall time: 6.52 s


In [12]:
%%time

query = 'Does App Service support iOS?'
js, md = qna(query)
display(Markdown(md))

intent: ["Does App Service support iOS?", "Can I deploy iOS apps on App Service?"]
answer: The retrieved documents do not provide any information related to whether App Service supports iOS. The requested information is not available in the retrieved data. Please try another query or topic.


The retrieved documents do not provide any information related to whether App Service supports iOS. The requested information is not available in the retrieved data. Please try another query or topic.

CPU times: user 11.3 ms, sys: 426 µs, total: 11.8 ms
Wall time: 4.46 s


In [16]:
%%time

query = 'How to config App service to reference Key Vault secrets in application settings?'
js, md = qna(query)
display(Markdown(md))

intent: ["How to reference Key Vault secrets in App Service application settings?", "Configure App Service to use Key Vault secrets in application settings"]
answer: To configure Azure App Service to reference Key Vault secrets in application settings, you need to first create a Key Vault and grant your app access to it[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-key-vault-references.md). Then, you can use the Key Vault reference as the value of app settings or connection strings in your App Service or Azure Functions apps[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-key-vault-references.md). To do this, you can use the `@Microsoft.KeyVault` syntax to reference the secret in the Key Vault[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-key-vault-references.md). You can create the app setting with the Key Vault reference using the Azure CLI command `az web

To configure Azure App Service to reference Key Vault secrets in application settings, you need to first create a Key Vault and grant your app access to it[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-key-vault-references.md). Then, you can use the Key Vault reference as the value of app settings or connection strings in your App Service or Azure Functions apps[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-key-vault-references.md). To do this, you can use the `@Microsoft.KeyVault` syntax to reference the secret in the Key Vault[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-key-vault-references.md). You can create the app setting with the Key Vault reference using the Azure CLI command `az webapp config appsettings set`[2](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/configure-encrypt-at-rest-using-cmk.md). After updating the app setting, the web app will restart and you can browse to it to ensure it has started correctly using the Key Vault reference[2](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/configure-encrypt-at-rest-using-cmk.md).

CPU times: user 7.67 ms, sys: 4.51 ms, total: 12.2 ms
Wall time: 6.99 s


In [17]:
%%time

query = 'How to deploy container from ACR to web app?'
js, md = qna(query)
display(Markdown(md))

intent: ["How to deploy container from ACR to web app?", "Deploy container from ACR to web app", "Steps to deploy container from ACR to web app"]
answer: To deploy a container from Azure Container Registry (ACR) to a web app, you can use the build, push, and deploy workflow provided by GitHub Actions[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-sql-github-actions.md). This workflow builds a container with the latest app changes, pushes the container to ACR, and updates the web application staging slot to point to the latest container pushed[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-sql-github-actions.md). The workflow includes a build and deploy job, where the build job checks out source code, authenticates with ACR, builds a container image, and deploys it to ACR[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-sql-github-actions.md). The deployment job

To deploy a container from Azure Container Registry (ACR) to a web app, you can use the build, push, and deploy workflow provided by GitHub Actions[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-sql-github-actions.md). This workflow builds a container with the latest app changes, pushes the container to ACR, and updates the web application staging slot to point to the latest container pushed[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-sql-github-actions.md). The workflow includes a build and deploy job, where the build job checks out source code, authenticates with ACR, builds a container image, and deploys it to ACR[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-sql-github-actions.md). The deployment job logs into Azure, gathers environment and Azure resource information, updates Web App Settings, and deploys to an App Service staging slot[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-sql-github-actions.md). To run the build, push, and deploy workflow, you need to open your `build-deploy.yaml` file in `.github/workflows` within your repository, verify that the environment variables for `AZURE_RESOURCE_GROUP` and `WEB_APP_NAME` match the ones in `azuredeploy.yaml`, and update the `ACR_LOGIN_SERVER` value for your ACR login server[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-sql-github-actions.md).

CPU times: user 8.5 ms, sys: 3.55 ms, total: 12.1 ms
Wall time: 6.98 s


In [19]:
query = 'ACR에 있는 컨테이너를 웹앱에 배포하는 방법에 대해서 알려줘?'
js, md = qna(query)
display(Markdown(md))

intent: ["ACR 컨테이너 웹앱 배포 방법", "ACR 컨테이너를 웹앱에 배포하는 방법", "ACR에서 웹앱으로 컨테이너 배포하는 방법"]
answer: GitHub Actions를 사용하여 ACR(Azure Container Registry)에 있는 컨테이너를 Azure Web App에 배포하는 방법은 다음과 같습니다[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-sql-github-actions.md)[2](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/deploy-ci-cd-custom-container.md):

1. GitHub Actions workflow 파일을 생성합니다.
2. Workflow 파일에서 Azure Container Registry에 로그인하고, 컨테이너 이미지를 빌드하고, ACR에 푸시합니다.
3. Azure Web App에 로그인하고, Web App 설정을 업데이트하고, 스테이징 슬롯에 배포합니다.
4. 커스텀 스크립트를 실행하여 SQL 데이터베이스를 업데이트하고, 스테이징 슬롯을 프로덕션으로 스왑합니다.

자세한 내용은 [1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-sql-github-actions.md)과 [2](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/deploy-ci-cd-custom-container.md)를 참조하시기 바랍니다.


GitHub Actions를 사용하여 ACR(Azure Container Registry)에 있는 컨테이너를 Azure Web App에 배포하는 방법은 다음과 같습니다[1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-sql-github-actions.md)[2](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/deploy-ci-cd-custom-container.md):

1. GitHub Actions workflow 파일을 생성합니다.
2. Workflow 파일에서 Azure Container Registry에 로그인하고, 컨테이너 이미지를 빌드하고, ACR에 푸시합니다.
3. Azure Web App에 로그인하고, Web App 설정을 업데이트하고, 스테이징 슬롯에 배포합니다.
4. 커스텀 스크립트를 실행하여 SQL 데이터베이스를 업데이트하고, 스테이징 슬롯을 프로덕션으로 스왑합니다.

자세한 내용은 [1](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/app-service-sql-github-actions.md)과 [2](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/app-service/deploy-ci-cd-custom-container.md)를 참조하시기 바랍니다.

## Grounding

In [18]:
%%time

query = "who is Micheal Jackson?"
reponse = qna(query)

intent: ["Who is Micheal Jackson?", "Tell me about Micheal Jackson", "Micheal Jackson biography"]
answer: The requested information is not available in the retrieved data. Please try another query or topic.
CPU times: user 10.6 ms, sys: 0 ns, total: 10.6 ms
Wall time: 4.03 s
